In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
from __future__ import print_function, division
current_dir = os.getcwd()
data_dir = current_dir + '/data/fastai/state/'
data_dir

'/Users/trinakarmakar/anaconda2/data/fastai/state/'

In [2]:
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

Using Theano backend.


In [3]:
batch_size = 64

In [ ]:
%cd $data_dir
%mkdir state
%cd state
%mkdir train
%mkdir sample
%mkdir sample/train
%mkdir sample/valid

In [4]:
sample_path = data_dir + 'sample/'

In [5]:
train_path = data_dir + 'train/'
%cd $train_path
%pwd

/Users/trinakarmakar/anaconda2/data/fastai/state/train


u'/Users/trinakarmakar/anaconda2/data/fastai/state/train'

In [ ]:
for d in glob('c?'):
    os.mkdir('../sample/train/'+d)
    os.mkdir('../sample/valid/'+d)

In [5]:
from shutil import copyfile

In [ ]:
g = glob('c?/*.jpg')
shuf = np.random.permutation(g)
for i in range(1500): copyfile(shuf[i], '../sample/train/' + shuf[i])

In [ ]:
%pwd

In [ ]:
g = glob('c?/*.jpg')
shuf = np.random.permutation(g)
for i in range(1000): copyfile(shuf[i], '../sample/valid/' + shuf[i])

In [ ]:
%cd ..

In [ ]:
%mkdir results

In [ ]:
%mkdir sample/test

In [5]:
#Create Batches:
batches = get_batches(data_dir+'sample/train', batch_size=batch_size)
val_batches = get_batches(data_dir+'sample/valid', batch_size=batch_size*2, shuffle=False)
test_batches = get_batches(data_dir+'sample/test', batch_size=batch_size, shuffle=False)

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 2000 images belonging to 1 classes.


In [6]:
(val_classes, trn_classes, val_labels, trn_labels, val_filenames, filenames,
    test_filename) = get_classes(sample_path)

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 2000 images belonging to 1 classes.


Basic Linear Model: First we will use a basic model with default parameter 
and we will use trick like using batchnorm so that we don't have normalize input data by hand.' 

In [ ]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(10, activation='softmax')
    ])

In [ ]:
model.compile(Adam(), loss = 'categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, 
                    validation_data = val_batches, nb_val_samples = val_batches.nb_sample)

We can see parameters and check if there is enough parameters to make sure we are not having high bias.

In [ ]:
model.summary()

Over 150K parameters no way we are having high bias problem.

In [ ]:
print('Number of paramaters:', 3*224*224)

Since we have a simple model with no regularization and plenty of parameters, it seems most likely that our learning rate is too high. Perhaps it is jumping to a solution where it predicts one or two classes with high confidence, so that it can give a zero prediction to as many classes as possible - that's the best approach for a model that is no better than random, and there is likely to be where we would end up with a high learning rate. So let's check:

In [ ]:
np.round(model.predict_generator(batches, batches.N)[:10],2)

Looks like our assumption is correct of high learning rate as it is almost always predicting 1 or 6

In [ ]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(10, activation='softmax')
    ])

model.compile(Adam(lr=1e-5), loss = 'categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, 
                    validation_data = val_batches, nb_val_samples = val_batches.nb_sample)

Yup we found our way out of the hole now lets increase the learning rate and see what happens.

In [ ]:
model.optimizer.lr = 0.001

In [ ]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=4, 
                    validation_data = val_batches, nb_val_samples = val_batches.nb_sample)

We are improving validation accuracy but we are not there yet, so we need to see 
if the validation data set enough large to give us consistent result.

In [ ]:
random_batches = get_batches(data_dir+'/sample/valid', batch_size=batch_size*2)

In [ ]:
val_res = [model.evaluate_generator(random_batches, random_batches.nb_sample) for i in range(10)]

In [ ]:
np.round(val_res, 2)

We see improvement and thats pretty constant - we can attribute that to the sample size

We saw that model is overfitting a lot but we can't use dropout for one layer NN we can use 
L2 regularization.

In [ ]:
model.optimizer.lr = 0.001

In [ ]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=5, 
                    validation_data = val_batches, nb_val_samples = val_batches.nb_sample)

We may be doing better but still its a long road ahead - Let's add a hidden layer and see

In [ ]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=[3,224,224]),
        Flatten(),
        Dense(100, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='softmax')
   ])
model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, 
                    validation_data=val_batches, nb_val_samples = val_batches.nb_sample)

In [ ]:
model.optimizer.lr = 0.001
model.fit_generator(batches, batches.nb_sample, nb_epoch=5, 
                    validation_data=val_batches, nb_val_samples = val_batches.nb_sample)

Doesn't look very promising and no wonder why CNN is state of art for computer vision

In [ ]:
def convl(batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(100, activation='softmax'),
            BatchNormalization(),
            Dense(10, activation='softmax')
    ])
    model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, batches.nb_sample, nb_epoch = 2, 
                    validation_data = val_batches, nb_val_samples= val_batches.nb_sample)
    model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, batches.nb_sample, nb_epoch = 5, 
                    validation_data = val_batches, nb_val_samples= val_batches.nb_sample)
    return model

In [ ]:
convl(batches)

We see training accuracy is reaching very high but not validation accuracy - so we have overfitting problem and
need to regularize parameters - Let's start by using data augmentation.

Data augmentation
To find the best data augmentation parameters, we can try each type of data augmentation, one at a time. For each type, we can try four very different levels of augmentation, and see which is the best. In the steps below we've only kept the single best result we found. We're using the CNN we defined above, since we have already observed it can model the data quickly and accurately.
Width shift: move the image left and right -

In [ ]:
gen_t = image.ImageDataGenerator(width_shift_range=0.1)
batches = get_batches(data_dir+'/train', gen_t, batch_size=batch_size)

In [ ]:
model = convl(batches)

In [ ]:
gen_t = image.ImageDataGenerator(height_shift_range=0.05) # Use height shift to move image up and down
batches = get_batches(data_dir+'/train', gen_t, batch_size=batch_size)

In [ ]:
#Change shear angles
gen_t = image.ImageDataGenerator(shear_range=0.1)
batche = get_batches(data_dir+'/train', gen_t, batch_size=batch_size)

In [ ]:
model = convl(batches)

In [ ]:
#rotation max in degrees
gen_t = image.ImageDataGenerator(rotation_range=15)
batches = get_batches(data_dir+'/train', gen_t, batch_size=batch_size)

In [ ]:
model = conv(batches)

In [ ]:
#Channel shift - randomly change color R, G, B
gen_t = image.ImageDataGenerator(channel_shift_range=20)
batches = get_batches(data_dir+'/train', gen_t, batch_size=batch_size)

In [ ]:
model = convl(batches)

In [ ]:
#Now putting it all together
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(data_dir+'/train', gen_t, batch_size=batch_size)

In [ ]:
model = convl(batches)

In [ ]:
model.optimizer.lr = 0.0001
model.fit_generator(batches, batches.nb_sample, nb_epoch=5,
                   validation_data = val_batches, nb_val_samples = val_batches.nb_sample)

In [ ]:
#We are making some progress - lets keep going 
model.fit_generator(batches, batches.nb_sample, nb_epoch=25,
                   validation_data = val_batches, nb_val_samples = val_batches.nb_sample)

We have shown progress in validation accuracy with some simple methods. For improving further we need to reduce overfitting 
a great deal but for that we need to use more data (original data not sample) to use dropout as dropout and large dataset works together.

In [8]:
def convl(batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(200, activation='softmax'),
            BatchNormalization(),
            Dropout(0.5),
            Dense(200, activation='softmax'),
            BatchNormalization(),
            Dropout(0.5),
            Dense(10, activation='softmax')
    ])
    model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, batches.nb_sample, nb_epoch = 2, 
                    validation_data = val_batches, nb_val_samples= val_batches.nb_sample)
    model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, batches.nb_sample, nb_epoch = 10, 
                    validation_data = val_batches, nb_val_samples= val_batches.nb_sample)
    return model

In [ ]:
model = convl(batches)

Epoch 1/2
1500/1500 [==============================] - 1008s - loss: 3.0835 - acc: 0.1373 - val_loss: 2.3026 - val_acc: 0.1060

In [37]:
%cd $data_dir

/Users/trinakarmakar/anaconda2/data/fastai/state


In [38]:
%mkdir test

In [40]:
%cd ../../../../downloads/test/

/Users/trinakarmakar/Downloads/test


In [44]:
%pwd

u'/Users/trinakarmakar/Downloads/test'

In [45]:
g = glob('img_*.jpg')
shuf = np.random.permutation(g)
for i in range(2000): copyfile(shuf[i], '../../anaconda2/data/fastai/state/sample/test/' + shuf[i])

In [47]:
trn = get_data(sample_path+'train')
val = get_data(sample_path+'valid')

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [49]:
save_array(data_dir+'results/val.dat', val)
save_array(data_dir+'results/trn.dat', trn)

In [51]:
val = load_array(data_dir+'results/val.dat')
trn = load_array(data_dir+'results/trn.dat')

Re-run sample experiments on full dataset


We should find that everything that worked on the sample (see statefarm-sample.ipynb), works on the full dataset too. Only better! Because now we have more data. So let's see how they go - the models in this section are exact copies of the sample notebook models.
Single conv layer

In [53]:
def conv1(batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    return model

In [56]:
model = conv1(batches)

Epoch 1/2
1500/1500 [==============================] - 339s - loss: 1.6419 - acc: 0.5107 - val_loss: 1.8621 - val_acc: 0.3310

In [58]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(sample_path+'train', gen_t, batch_size=batch_size)

Found 1500 images belonging to 10 classes.


In [59]:
model.optimizer.lr = 0.0001
model.fit_generator(batches, batches.nb_sample, nb_epoch=15, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/15
1500/1500 [==============================] - 335s - loss: 1.7961 - acc: 0.4333 - val_loss: 2.1008 - val_acc: 0.3410

I'm shocked by *how* good these results are! We're regularly seeing 75-80% accuracy on the validation set, which puts us into the top third or better of the competition. With such a simple model and no dropout or semi-supervised learning, this really speaks to the power of this approach to data augmentation.

Unfortunately, the results are still very unstable - the validation accuracy jumps from epoch to epoch. Perhaps a deeper model with some dropout would help.

In [61]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(sample_path+'train', gen_t, batch_size=batch_size)

Found 1500 images belonging to 10 classes.


In [62]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Convolution2D(32,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(64,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(128,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Flatten(),
        Dense(200, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(200, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])

In [63]:
model.compile(Adam(lr=10e-5), loss='categorical_crossentropy', metrics=['accuracy'])

In [64]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1500/1500 [==============================] - 625s - loss: 3.4998 - acc: 0.1260 - val_loss: 2.2730 - val_acc: 0.1590

In [65]:
model.optimizer.lr=0.001

In [ ]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=10, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/10
1500/1500 [==============================] - 903s - loss: 2.9026 - acc: 0.1900 - val_loss: 2.2194 - val_acc: 0.1760

In [ ]:
model.optimizer.lr=0.00001

In [ ]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=10, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

This is looking quite a bit better - the accuracy is similar, but the stability is higher. There's still some way to go however...

Since we have so little data, and it is similar to imagenet images (full color photos), using pre-trained VGG weights is likely to be helpful - in fact it seems likely that we won't need to fine-tune the convolutional layer weights much, if at all. So we can pre-compute the output of the last convolutional layer, as we did in lesson 3 when we experimented with dropout. (However this means that we can't use full data augmentation, since we can't pre-compute something that changes every image.)

Imagenet conv features:

Since we have so little data, and it is similar to imagenet images (full color photos), using pre-trained VGG weights is likely to be helpful - in fact it seems likely that we won't need to fine-tune the convolutional layer weights much, if at all. So we can pre-compute the output of the last convolutional layer, as we did in lesson 3 when we experimented with dropout. (However this means that we can't use full data augmentation, since we can't pre-compute something that changes every image.)

In [7]:
vgg = Vgg16()

In [8]:
model = vgg.model

In [9]:
last_conv_layer_idx = [i for i,l in enumerate(model.layers) if type(l) is Convolution2D][-1]
conv_layers = model.layers[:last_conv_layer_idx+1]

In [10]:
conv_model = Sequential(conv_layers)

In [11]:
(val_classes, trn_classes, val_labels, trn_labels, 
             val_filenames, filenames, test_filenames) = get_classes(sample_path)

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 2000 images belonging to 1 classes.


In [12]:
conv_feature = conv_model.predict_generator(batches, batches.nb_sample)
conv_val_feature = conv_model.predict_generator(val_batches, val_batches.nb_sample)
conv_test_feature = conv_model.predict_generator(test_batches, test_batches.nb_sample)

In [17]:
save_array(data_dir+'results/conv_val_feat.dat', conv_val_feature)
save_array(data_dir+'results/conv_test_feat.dat', conv_test_feature)
save_array(data_dir+'results/conv_feat.dat', conv_feature)

In [53]:
conv_feature = load_array(data_dir+'results/conv_feat.dat')
conv_val_feature = load_array(data_dir+'results/conv_val_feat.dat')
conv_val_feature.shape

(1000, 512, 14, 14)

Batchnorm dense layers on pretrained conv layers.

Now we can create a model with additional layers (after the pre computed part) and use weight saved from the pre computed
part to train a new model.

In [13]:
def get_bn_da_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')
        ]

In [14]:
p = 0.8

In [15]:
bn_model = Sequential(get_bn_da_layers(p))
bn_model.compile(Adam(lr=0.0001), loss = 'categorical_crossentropy', metrics=['accuracy'])

In [16]:
bn_model.fit(conv_feature, trn_labels, batch_size=batch_size, nb_epoch=1, 
             validation_data=(conv_val_feature, val_labels))

Train on 1500 samples, validate on 1000 samples
Epoch 1/1
1500/1500 [==============================] - 7s - loss: 5.0504 - acc: 0.1027 - val_loss: 2.5781 - val_acc: 0.0980

In [17]:
bn_model.optimizer.lr = 0.01

In [19]:
bn_model.fit(conv_feature, trn_labels, batch_size=batch_size, nb_epoch=2, 
             validation_data=(conv_val_feature, val_labels))

Train on 1500 samples, validate on 1000 samples
Epoch 1/2
1500/1500 [==============================] - 4s - loss: 4.9863 - acc: 0.1107 - val_loss: 2.3941 - val_acc: 0.1020 128/1500 [=>............................] - ETA: 3s - loss: 5.2127 - acc: 0.0703

In [61]:
bn_model.save_weights(data_dir+'models/conv8.h5')

Lets experiment dropout with augmentation with pre computed model

In [33]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
da_batches = get_batches(sample_path+'train', gen_t, batch_size=batch_size, shuffle=False)

Found 1500 images belonging to 10 classes.


In [34]:
#use this to create dataset of convolutional 5 times larger than the training set
da_conv_feature = conv_model.predict_generator(da_batches, da_batches.nb_sample*5)

In [35]:
save_array(data_dir+'results/da_conv_feature2.dat', da_conv_feature)

In [36]:
da_conv_feature = load_array(data_dir+'results/da_conv_feature2.dat')

In [38]:
da_conv_feat = np.concatenate([da_conv_feature, conv_feature])

In [39]:
da_trn_labels = np.concatenate([trn_labels]*6)

Based on findings from previous experiement lets create bigger dense layer 

In [40]:
def get_bn_da_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')
    ]

In [41]:
p = 0.8

In [42]:
bn_model = Sequential(get_bn_da_layers(p))
bn_model.compile(Adam(lr=0.001), loss = 'categorical_crossentropy', metrics=['accuracy'])

Now we can train the model with pre computed model and augmented data 

In [43]:
bn_model.fit(da_conv_feature, da_trn_labels, batch_size=batch_size, nb_epoch=1, 
             validation_data=(conv_val_feature, val_labels))

Exception: Input arrays should have the same number of samples as target arrays. Found 7500 input samples and 9000 target samples.

In [ ]:
bn_model.optimizer.lr = 0.01

In [ ]:
bn_model.fit(da_conv_feature, da_trn_labels, batch_size=batch_size, nb_epoch=1,
            validation_data = (conv_val_feature, val_labels))

In [ ]:
bn_model.optimizer.lr = 0.0001

In [ ]:
bn_model.fit(da_conv_feature, da_trn_labels, batch_size=batch_size, nb_epoch=1,
            validation_data = (conv_val_feature, val_labels))

In [ ]:
bn_model.save_weights(path+'models/da_conv8_1.h5')

Pseudo Labeling:
We will try to use pseudo labaleing and knowledge distillation to understand semi supervised learning. For this we will validation set as unlabeled data without the need for test dataset and later we will use test data set. 

In [ ]:
#lets find out prediction of our model
val_pseudo = bn_model.predict(conv_val_feature, batch_size=batch_size)
#Concatenate that with training label
combo_label = np.concatenate([da_trn_labels, val_pseudo])
combo_feat = np.concatenate([da_conv_feat, conv_val_feat])

..and fine tune model using that data

In [ ]:
bn_model.load_weights(data_dir+'models/da_conv8_1.h5')

In [ ]:
bn_model.fit(combo_feat, combo_pseudo, batch_size=batch_size,nb_epoch=1, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_model.fit(combo_feat, combo_pseudo, batch_size=batch_size,nb_epoch=4, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_model.optimizer.lr = 0.00001
bn_model.fit(combo_feat, combo_pseudo, batch_size=batch_size,nb_epoch=4, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_model.save_weights(data_dir+'models/bn-ps8.h5')

Submit:
    
we'll find a good clipping amount using the validation set prior to submissing

In [ ]:
def co-clip(arr, mx): return np.clip(arr, (1-mx)/9, mx)

In [ ]:
keras.metrics.categorical_crossentropy(val_labels, do_clip(val_preds, 0.93)).eval()

In [ ]:
conv_test_feat = load-array(data_dir+'results/conv_test_feat.dat')

In [ ]:
preds = bn_model.predict(conv_test_feat, batch_size=batch_size*2)

In [ ]:
subm = do_clip(preds, 0.93)

In [ ]:
subm_name = data-dir+'results/subm.gz'

In [ ]:
classes = sorted(batches.class_indices, key=batches.class_indices.get)

In [ ]:
submission = pd.Dataframe(subm, columns=classes)
submission.insert(0, 'img', [a[4:] for a in test_filenames])
submission.head()

In [ ]:
submission.to_csv(subm_name, index=False, compression='gzip')

In [ ]:
FileLink(subm_name)